# MOQ Price Analysis for Wholesale B2B Businesses

## importing libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

## extracting the dataset

In [ ]:
df = pd.read_csv('online_retail_II.csv')
print(df.dtypes)

## analysing data quality

In [ ]:
# finding missing data
missing_data = pd.DataFrame({
    'columns': df.columns,
    'null count': df.isnull().sum(),
    'null percentage': (df.isnull().sum() / len(df) * 100).round(2)
})

print(missing_data)

In [ ]:
# finding unique data
for col in df.columns:
    print(f'{col}: {df[col].nunique()}')

In [ ]:
# getting a sample of the dataset
df.head(10)

In [ ]:
# descriptive statistics from the dataset
df.describe()

## identifying and documenting problems

In [ ]:
# negative numbers in quantity
negative_qty = df[df['Quantity'] < 0]
print(negative_qty.head(10))

In [ ]:
# zero or negative prices in price
zero_price = df[df['Price'] <= 0]
print(zero_price.head(10))

In [ ]:
# special codes (non-products)
special_codes = df[df['StockCode'].str.contains('^[A-Z]+$', na=False, regex=True)]
print(special_codes['StockCode'].value_counts().head(10))

In [ ]:
# missing customer id
no_customer = df[df['Customer ID'].isnull()]
print(no_customer.head(10))

In [ ]:
# missing descriptions
no_desc = df[df['Description'].isnull()]
print(no_desc.head(10))

In [ ]:
# finding outliers in quantity
print(f"minimun quantity: {df['Quantity'].min()}")
print(f"maximun quantity: {df['Quantity'].max()}")
print(f"mean quantity: {df['Quantity'].mean()}")
print(f"99 percentile quantity: {df['Quantity'].quantile(0.99)}")
print(f"99.9 percentile quantity: {df['Quantity'].quantile(0.999)}")

## cleaning the dataset

In [ ]:
# getting a copy for the process
df_clean = df.copy()
print(f"number of initial records: {len(df_clean)}")

In [ ]:
# cleaning report
cleaning_report = {
    'step': [],
    'description': [],
    'removed count': [],
    'remaining count': []
}

def add_cleaning_step(step_name, description, removed_count, remaining_count):
    cleaning_report['step'].append(step_name)
    cleaning_report['description'].append(description)
    cleaning_report['removed count'].append(removed_count)
    cleaning_report['remaining count'].append(remaining_count)

### step 1: removing the returns

In [ ]:
# counting before removing
before_q = len(df_clean)

# filtering
df_clean = df_clean[df_clean['Quantity'] > 0]

# counting after removing
after_q = len(df_clean)
removed_q = before_q - after_q

print(f"removed: {removed_q} records")
print(f"remaining: {after_q} records")

# adding to report
add_cleaning_step('step1', 'removed negative quantities', removed_q, after_q)

### step2: removing invalid prices

In [ ]:
before_p = len(df_clean)

# removing negative or zero prices
df_clean = df_clean[df_clean['Price'] > 0]

after_p = len(df_clean)
removed_p = before_p - after_p

print(f"removed: {removed_p} records")
print(f"remaining: {after_p} records")

add_cleaning_step('step2', 'removed negative and zero prices', removed_p, after_p)

### step3: removing non-product stock codes

In [ ]:
before_s = len(df_clean)

# identifying special code patterns
special_patterns = ['POST', 'D', 'DOT', 'M', 'BANK CHARGES', 'PADS', 'C2', 'CRUK', 'AMAZONFEE']

# removing stock codes with special patterns
df_clean = df_clean[~(df_clean['StockCode'].isin(special_patterns))]

# removing stock codes with one worded codes
df_clean = df_clean[~(df_clean['StockCode'].str.match('^[A-Z]$', na=False))]

after_s = len(df_clean)
removed_s = before_s - after_s

print(f"removed: {removed_s} records")
print(f"remaining: {after_s} records")

add_cleaning_step('step3', 'removed non-product stock codes', removed_s, after_s)

### step4: removing missing descriptions

In [ ]:
before_d = len(df_clean)

df_clean = df_clean[df_clean['Description'].notna()]

after_d = len(df_clean)
removed_d = before_d - after_d

print(f"removed: {removed_d} records")
print(f"remaining: {after_d} records")

add_cleaning_step('step4', 'removed missing descriptions', removed_d, after_d)

### step5: removing no customer id values

In [ ]:
# finding records with no customer ids
no_customer = df_clean[df_clean['Customer ID'].isna()]

before_c = len(df_clean)

# removing no customer id records for moq strategy
df_clean = df_clean[df_clean['Customer ID'].notna()]

after_c = len(df_clean)
removed_c = before_c - after_c

print(f"removed: {removed_c} records")
print(f"remaining: {after_c} records")

add_cleaning_step('step5', 'removed records with no customer id', removed_c, after_c)

### step6: managing outliers in quantity with IQR method

In [ ]:
# calculating IQR
Q1 = df_clean['Quantity'].quantile(0.25)
Q3 = df_clean['Quantity'].quantile(0.75)
IQR = Q3 - Q1

# defining bounds with (3*IQR) for extreme outliers
lower_bound = Q1 - 3 * IQR
upper_bound = Q3 + 3 * IQR

print('quantity stats')
print(f'Q1: {Q1}')
print(f'Q3: {Q3}')
print(f'IQR: {IQR}')
print(f'accepted bounds are from {lower_bound} to {upper_bound}')

In [ ]:
# identifying outliers
outliers = df_clean[(df_clean['Quantity'] < lower_bound) | (df_clean['Quantity'] > upper_bound)]

before_o = len(df_clean)

# note: order quantities more than 10000 are considered extreme and are removed
df_clean = df_clean[df_clean['Quantity'] <= 10000]

after_o = len(df_clean)
removed_o = before_o - after_o

print(f"removed: {removed_o} records")
print(f"remaining: {after_o} records")

add_cleaning_step('step6', 'removed outliers with IQR method', removed_o, after_o)

### step7: cleaning and standardizing descriptions

In [ ]:
# making all texts uppercase
df_clean['Description'] = df_clean['Description'].str.upper()

# removing extra spaces
df_clean['Description'] = df_clean['Description'].str.strip()

# removing extra special characters
df_clean['Description'] = df_clean['Description'].str.replace('[^\w\s]', ' ', regex=True)

# removing extra spaces
df_clean['Description'] = df_clean['Description'].str.replace('\s+', ' ', regex=True)

add_cleaning_step('step7', 'standardized descriptions', 0, after_d)

### step8: standardizing datetimes

In [ ]:
# checking datetime
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])

# extracting time from datetime
df_clean['Year'] = df_clean['InvoiceDate'].dt.year
df_clean['Month'] = df_clean['InvoiceDate'].dt.month
df_clean['Day'] = df_clean['InvoiceDate'].dt.day
df_clean['DayOfWeek'] = df_clean['InvoiceDate'].dt.dayofweek
df_clean['Hour'] = df_clean['InvoiceDate'].dt.hour

# making a quarter
df_clean['Quarter'] = df_clean['InvoiceDate'].dt.quarter

# making a season
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

df_clean['Season'] = df_clean['Month'].apply(get_season)

# invoice date ranges
print(f"invoice date range is from {df_clean['InvoiceDate'].min()} to {df_clean['InvoiceDate'].max()}")

add_cleaning_step('step8', 'fixed datetime in invoice dates', 0, 0)

### step9: correcting data types

In [ ]:
# turning all customer ids to int
df_clean['Customer ID'] = df_clean['Customer ID'].astype(int)

# turning stock codes to strings
df_clean['StockCode'] = df_clean['StockCode'].astype(str)

# turning invoices to strings
df_clean['Invoice'] = df_clean['Invoice'].astype(str)

# making price float and quantity int
df_clean['Quantity'] = df_clean['Quantity'].astype(int)
df_clean['Price'] = df_clean['Price'].astype(float)

add_cleaning_step('step9', 'corrected data types', 0, 0)

### step10: generating the total price column

In [ ]:
df_clean['TotalPrice'] = (df_clean['Quantity'] * df_clean['Price']).round(2)
df_clean['TotalPrice']

### step11: removing duplicates (if existing)

In [ ]:
before_u = len(df_clean)

# removing duplicates
df_clean = df_clean.drop_duplicates()

after_u = len(df_clean)
removed_u = before_u - after_u

if removed_u > 0:
    add_cleaning_step('step11', 'removed existing duplicates', removed_u, after_u)


### step12: final checks and validation

In [ ]:
# checking for null values
null_check = df_clean.isnull().sum()
print(f"null count: {null_check}")

In [ ]:
# checking for negative values
negative_qty = (df_clean['Quantity'] < 0).sum()
negative_price = (df_clean['Price'] <= 0).sum()
print(negative_qty, negative_price)

In [ ]:
# final stats
print(f"number of records: {len(df_clean)}")
print(f"number of unique customers: {df_clean['Customer ID'].nunique()}")
print(f"number of unique products: {df_clean['StockCode'].nunique()}")
print(f"number of unique invoices: {df_clean['Invoice'].nunique()}")
print(f"date range: {df_clean['InvoiceDate'].min()} -> {df_clean['InvoiceDate'].max()}")

In [ ]:
# key stats
print("\nQuantity")
print(df_clean['Quantity'].describe())
print("\nPrice")
print(df_clean['Price'].describe())
print("\nTotalPrice")
print(df_clean['TotalPrice'].describe())

### step13: documenting cleaning report

In [ ]:
cleaning_df = pd.DataFrame(cleaning_report)

total_removed = df.shape[0] - df_clean.shape[0]
removal_percentage = (total_removed / df.shape[0]) * 100

print(cleaning_df.to_string(index=False))
print(f"\nnumber of initial records: {df.shape[0]}")
print(f"number of final records: {df_clean.shape[0]}")
print(f"total of removed records: {total_removed} ({removal_percentage:.1f}%)")
print(f"total of kept records: {df_clean.shape[0]} ({100 - removal_percentage:.1f}%)")

### step14: exporting the cleaned data

In [ ]:
# saving cleaned data to csv
df_clean.to_csv('online_retail_cleaned.csv', index=False, encoding='utf-8-sig')

# for faster importing
df_clean.to_pickle('online_retail_cleaned.pkl')

### step15: validation charts

In [ ]:
# validating if the extreme outliers are deleted, negative values exist and if the distrobution is normal and logical
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

fig, axes = plt.subplots(2, 2, figsize=(16,12))
fig.suptitle('Data Cleaning Validation Charts', fontsize=16, fontweight='bold')

# 1) Quantity distrobution chart
ax1 = axes[0, 0]
ax1.hist(df_clean['Quantity'], bins=50, edgecolor='black', alpha=0.7)
ax1.set_xlabel('Quantity', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.set_title('Distribution of Quantity (After Cleaning)', fontsize=14, fontweight='bold')
ax1.axvline(df_clean['Quantity'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df_clean["Quantity"].mean():.2f}')
ax1.axvline(df_clean['Quantity'].median(), color='green', linestyle='--', linewidth=2, label=f'Median: {df_clean["Quantity"].median():.2f}')
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3)

# adding inset to box plot
axins1 = inset_axes(ax1, width="40%", height="30%", loc='upper right')
axins1.boxplot(df_clean['Quantity'], vert=False)
axins1.set_xlabel('Quantity', fontsize=8)
axins1.set_title('Box Plot', fontsize=9)


# Price distrobution chart
ax2 = axes[0, 1]
ax2.hist(df_clean['Price'], bins=50, edgecolor='black', alpha=0.7, color='orange')
ax2.set_xlabel('Price (£)', fontsize=12)
ax2.set_ylabel('Frequency', fontsize=12)
ax2.set_title('Distribution of Price (After Cleaning)', fontsize=14, fontweight='bold')
ax2.axvline(df_clean['Price'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: £{df_clean["Price"].mean():.2f}')
ax2.axvline(df_clean['Price'].median(), color='green', linestyle='--', linewidth=2, label=f'Median: £{df_clean["Price"].median():.2f}')
ax2.legend(loc='lower right')
ax2.grid(True, alpha=0.3)

# TotalPrice distrobution chart
ax3 = axes[1, 0]
total_price_99 = df_clean['TotalPrice'].quantile(0.99)
df_plot = df_clean[df_clean['TotalPrice'] <= total_price_99]
ax3.hist(df_plot['TotalPrice'], bins=50, edgecolor='black', alpha=0.7, color='green')
ax3.set_xlabel('TotalPrice (£)', fontsize=12)
ax3.set_ylabel('Frequency', fontsize=12)
ax3.set_title(f'Distribution of TotalPrice (up to 99th percentile: £{total_price_99:.2f})', fontsize=14, fontweight='bold')
ax3.axvline(df_clean['TotalPrice'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: £{df_clean["TotalPrice"].mean():.2f}')
ax3.axvline(df_clean['TotalPrice'].median(), color='green', linestyle='--', linewidth=2, label=f'Median: £{df_clean["TotalPrice"].median():.2f}')
ax3.legend(loc='lower right')
ax3.grid(True, alpha=0.3)

# number of transactions in time chart
ax4 = axes[1, 1]
daily_transactions = df_clean.groupby(df_clean['InvoiceDate'].dt.date).size()
ax4.plot(daily_transactions.index, daily_transactions.values, linewidth=1, color='purple')
ax4.set_xlabel('Date', fontsize=12)
ax4.set_ylabel('Number of Transactions', fontsize=12)
ax4.set_title('Transactions Over Time (Daily)', fontsize=14, fontweight='bold')
ax4.grid(True, alpha=0.3)

# rotating the x axes
for label in ax4.get_xticklabels():
    label.set_rotation(45)
    label.set_ha('right')

plt.tight_layout()

# saving the charts
chart_filename = 'data_cleaning_validation.png'
plt.savefig(chart_filename, dpi=300, bbox_inches='tight')

### step16: final validation test

In [ ]:
# Final validation Tests

# list of tests
tests_passed = []
tests_failed = []

# TEST: no nulls in customer id
test_1 = 'no NULLs in customer id'
try:
    assert df_clean['Customer ID'].isna().sum() == 0, "customer id contains nulls!"
    tests_passed.append(test_1)
    print(f"✓ PASS: {test_1}")
except AssertionError as e:
    tests_failed.append((test_1, str(e)))
    print(f"✗ FAIL: {test_1} - {e}")


# TEST: no negatives in quantity
test_2 = 'no negative quantity'
try:
    assert (df_clean['Quantity'] < 0).sum() == 0, "negative quantity found!"
    tests_passed.append(test_2)
    print(f"✓ PASS: {test_2}")
except AssertionError as e:
    tests_failed.append((test_2, str(e)))
    print(f"✗ FAIL: {test_2} - {e}")

# TEST: no negatives or zero prices
test_3 = 'no negative or zero prices'
try:
    assert (df_clean['Quantity'] < 0).sum() == 0, "negative quantity found!"
    tests_passed.append(test_3)
    print(f"✓ PASS: {test_3}")
except AssertionError as e:
    tests_failed.append((test_3, str(e)))
    print(f"✗ FAIL: {test_3} - {e}")

# TEST: TotalPrice calculated correctly
test_4 = 'correct total price'
try:
    assert (df_clean['Quantity'] < 0).sum() == 0, "negative quantity found!"
    tests_passed.append(test_4)
    print(f"✓ PASS: {test_4}")
except AssertionError as e:
    tests_failed.append((test_4, str(e)))
    print(f"✗ FAIL: {test_4} - {e}")

# TEST: no complete duplicates
test_5 = 'no complete duplicates'
try:
    assert (df_clean['Quantity'] < 0).sum() == 0, "negative quantity found!"
    tests_passed.append(test_5)
    print(f"✓ PASS: {test_5}")
except AssertionError as e:
    tests_failed.append((test_5, str(e)))
    print(f"✗ FAIL: {test_5} - {e}")

# TEST: quantity in accepted range
test_6 = 'quantity in accepted range'
try:
    assert df_clean['Quantity'].min() >= 1, f"minimum quantity is {df_clean['Quantity'].min()}"
    assert df_clean['Quantity'].max() <= 10000, f"maximum quantity is {df_clean['Quantity'].max()}"
    tests_passed.append(test_6)
    print(f"✓ PASS: {test_6}")
except AssertionError as e:
    tests_failed.append((test_6, str(e)))
    print(f"✗ FAIL: {test_6} - {e}")

# TEST: invoice in accepted range
test_7 = 'invoice in accepted range'
try:
    min_date = df_clean['InvoiceDate'].min()
    max_date = df_clean['InvoiceDate'].max()
    assert min_date >= pd.Timestamp('2009-01-01'), f"Earliest date {min_date} is before 2009"
    assert max_date <= pd.Timestamp('2012-01-01'), f"Latest date {max_date} is after 2011"
    tests_passed.append(test_7)
    print(f"✓ PASS: {test_7}")
except AssertionError as e:
    tests_failed.append((test_7, str(e)))
    print(f"✗ FAIL: {test_7} - {e}")

In [ ]:
# overall tests review
if tests_failed:
    print(f"Number of Failed Tests: {len(tests_failed)}")
    print(f"\nFailed Tests:")
    for test, error in tests_failed:
        print(f"test: {test}, error: {error}")
else:
    print("ALL TESTS PASSED")

## Checking Metrics for EDA

In [ ]:
# Loading the Cleaned DataFrame
df = pd.read_pickle('online_retail_cleaned.pkl')

# Calculating the invoice_level metrics
invoice_metrics = df.groupby('Invoice').agg({
    'TotalPrice': 'sum',
    'Quantity': 'sum'
})